In [1]:
# One Code (.py)

# Execution Function all code
def mainExecution():
    # import libraries
    import urllib.request
    import urllib.parse
    import tabula
    import numpy as np
    from bs4 import BeautifulSoup
    from selenium import webdriver
    import time
    import pandas as pd
    from pandas import ExcelWriter
    from pandas import ExcelFile
    import requests
    from webdriver_manager.chrome import ChromeDriverManager
    import glob
    
    # inisialisasi url dan driver
    urlpage = 'https://loker.bnpb.go.id/s/GugusTugasCovid19?path=%2FData%20Kemenkes%2FLaporan%20Media'
    print(urlpage)
    # jalankan chrome web driver dari sesuai path yang anda tuju
    driver = webdriver.Chrome(r'C:\Users\bader\Music\chromedriver_win32\chromedriver.exe')
    
    # dapatkan halaman web
    driver.get(urlpage)
    # eksekusi script untuk dari mensecrol ke bawah halaman
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")

    SCROLL_PAUSE_TIME = 0.5

    # mendapatkan kedalaman scroll
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll ke bawah
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # waktu pause ke bawah 
        time.sleep(SCROLL_PAUSE_TIME)

        # hitung bats scroll baru dan bandingkan dengan batasan scorll akhir
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


    # berhenti sejenak 30 detik
    time.sleep(30)
    # driver.quit()
    
    # Find elements by xpath
    # results = driver.find_elements_by_xpath("//*[@id='componentsContainer']//*[contains(@id,'listingsContainer')]//*[@class='product active']//*[@class='title productTitle']")
    # Updated 07 Juli 2020:
    results = driver.find_elements_by_xpath("//*[@id='fileList']/tr/td[2]/a")
    resultDate = driver.find_elements_by_xpath("//*[@id='fileList']/tr/td[4]/span")
    print('Number of results Info File :', len(results))
    print('Number of results Info Date :', len(resultDate))
    
    #Check and get all name file before downloading
    list_name_of_files = []
    list_of_files = glob.glob('hasil-pdf-download/*')
    for i in range(len(list_of_files)):
        list_name_of_files.append(list_of_files[i].split("\\")[1])

    # loop over results and download a file
    crawl = 0;
    for result in results:
        bnpb_link = result.get_attribute("href")
        name_pdf = result.find_element_by_class_name('innernametext').text
        ext_file = result.find_element_by_class_name('extension').text
        date_info = resultDate[crawl].get_attribute('data-original-title')
        time_left = resultDate[crawl].text

        print(date_info)

        if name_pdf+""+ext_file not in list_name_of_files:
            # Start to download dari url
            myfile = requests.get(bnpb_link)
            open('C:/Users/bader/Music/Percobaan/hasil-pdf-download/'+name_pdf+""+ext_file, 'wb').write(myfile.content)

            # Read File from Tabula Library
            if ext_file == ".pdf":
                df = tabula.read_pdf("C:/Users/bader/Music/Percobaan/hasil-pdf-download/"+name_pdf+""+ext_file, pages = 'all', lattice = True)
                if df is None:
                    pass # do nothing
                else:
                    # Replace each values that responding from NaN with blank / empty string
                    df = df.replace(np.nan, '')
                    # Replace each values that responding from \r into Space
                    df = df.replace('\r', ' ')
                    # Replace each unamed columns with blank string
                    df = df.rename(columns = {'Unnamed: 5':'','Unnamed: 6':'','Unnamed: 7':'','Unnamed: 8':'','Unnamed: 9':'','Unnamed: 10':''})

                    # Save into Excel
                    dfHeader = pd.DataFrame([list(df.columns)[0],list(df.columns)[1],list(df.columns)[2],'','',list(df.columns)[3]
                                                 ,'','',list(df.columns)[4],'','',''])

                    dfHeader = dfHeader.transpose()
                    dfData = pd.DataFrame(df)
                    writer = ExcelWriter("C:/Users/bader/Music/Percobaan/hasil-convert-download-excel/"+name_pdf+".xlsx")
                    dfHeader.to_excel(writer,'Sheet 1',index=False,header=False)
                    dfData.to_excel(writer,'Sheet 1',index=False,header=False,startrow=1)
                    writer.save()

                    # Save into CSV without preprocessing
                    tabula.convert_into("C:/Users/bader/Music/Percobaan/hasil-pdf-download/"+name_pdf+".pdf", "C:/Users/bader/Music/Percobaan/hasil-convert-download-csv/"+name_pdf+".csv",lattice = True, output_format="csv", pages='all')

                    # Save file after preprocessing date file into folder (hasil-convert-download-excel-preprocessing)
                    dfDataCurrent = pd.DataFrame(dfData.iloc[1:len(dfData)-2,:])

                    # Create Latitude and Longitude column
                    dfDataCurrent['Latitude'] = ""
                    dfDataCurrent['Longitude'] = ""

                    # Fill all data Latitude and Longitude
                    for i, row in dfDataCurrent.iterrows():
                        # Nama Provinsi setiap pengulang --> dfDataCurrent.iloc[:i,1]
                        address = dfDataCurrent.loc[i,dfDataCurrent.columns[1]]
                        # Buat Request ke sebuah url
                        url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(str(address)) +'?format=json'
                        # Ambil response berbentuk json
                        # Pada posisi index 0 dengan key lat --> response[0]["lat"]
                        # Pada posisi index 0 dengan key lon --> response[0]["lon"]
                        response = requests.get(url).json()
                        dfDataCurrent.loc[i,dfDataCurrent.columns[-1]] = response[0]["lon"];
                        dfDataCurrent.loc[i,dfDataCurrent.columns[-2]] = response[0]["lat"];

                    # Create date from date_release
                    indexStart = 1
                    newColumn = [f''+date_info for i in range(len(dfDataCurrent))]
                    dfDataCurrent.insert(loc=indexStart, column='Tanggal', value=newColumn)            

                    # Create Excel Preprocessing
                    dfDataCurrent = pd.DataFrame(dfDataCurrent.iloc[:len(dfDataCurrent),:])
                    writer = ExcelWriter("C:/Users/bader/Music/Percobaan/hasil-convert-download-excel-preprocessing/"+name_pdf+".xlsx")
                    dfHeader = pd.DataFrame([list(df.columns)[0],'Tanggal',list(df.columns)[1],list(df.columns)[2],'','',list(df.columns)[3]
                                                 ,'','',list(df.columns)[4],'','','Latitude','Longitude'])
                    dfHeader = dfHeader.transpose()
                    dfHeader.to_excel(writer,'Sheet 1',index=False,header=False)
                    dfDataCurrent.to_excel(writer,'Sheet 1',index=False,header=False,startrow=1)
                    writer.save()
            print('Yes It is, Please Download this file : '+(name_pdf+""+ext_file))
        else:
            print('Tidak Terdownload, karena file sudah ada')

        crawl=crawl+1
        
    # Close all focused web driver
    driver.quit()
    
    #Check and get all name file before downloading
    list_name_of_files = []
    list_of_files = glob.glob('hasil-pdf-download/*')
    for i in range(len(list_of_files)):
        list_name_of_files.append(list_of_files[i].split("\\")[1])
    
    if len(list_name_of_files) == len(results):
#         print("masuk")
        # Combining all data into into one file
        dfDataTotal = pd.DataFrame()
        list_name_of_files = []
        list_of_files = glob.glob('hasil-convert-download-excel-preprocessing/*')
        for i in range(len(list_of_files)):
            if dfDataTotal.empty:
                dfDataTotal = pd.read_excel('hasil-convert-download-excel-preprocessing/'+list_of_files[i].split("\\")[1])
            else:
                dfDataPreprocessing = pd.read_excel('hasil-convert-download-excel-preprocessing/'+list_of_files[i].split("\\")[1])
                dfDataPreprocessing = dfDataPreprocessing.dropna(axis=1, how='all')
                dfDataTotal = pd.DataFrame(np.concatenate((dfDataTotal.values,dfDataPreprocessing.values)),columns=dfDataTotal.columns)

        # Save File all concatenating excel file into one
        dfHeader = pd.DataFrame([list(dfDataTotal.columns)[0],list(dfDataTotal.columns)[1],list(dfDataTotal.columns)[2],'JK.sebelum','jk.penambahan'
                                ,'JK.kum','JK.sembuh_sebelum','JK.sembuh_penambahan','JK.sembuh_kum','JK.meninggal_sebelum','JK.meninggal_penambahan','JK.meninggal_kum','Latitude','Longitude','suspect_diisolasi'
                                                ,'suspect_discarded','closecontact_dikarantina','closecontact_discarded','confirmation_selesai'
                                                ,'confirmation_meninggal','suspect_meninggal','closecontact_meninggal','probable_meninggal'
                                                ,'confirmation','closecontact','closecontact','pertumbuhan_closecontact','pertumbuhan_closecontact_discarded'
                                                ,'pertumbuhan_closecontact_dikarantina','pertumbuhan_suspect','pertumbuhan_suspect_discarded'
                                                ,'pertumbuhan_suspect_diisolasi','pertumbuhan_probable','pertumbuhan_probable_discarded'
                                                ,'pertumbuhan_probable_diisolasi','pertumbuhan_probable_meninggal','pertumbuhan_confirmation'
                                                ,'pertumbuhan_confirmation_selesai','pertumbuhan_confirmation_meninggal','probable_diisolasi'
                                                ,'probable_discarded','probable'])

        dfHeader = dfHeader.transpose()
        writer = ExcelWriter("C:/Users/bader/Music/Percobaan/all-concate-excel-file.xlsx")
        dfHeader.to_excel(writer,'Sheet 1',index=False,header=False)
        dfDataTotal.to_excel(writer,'Sheet 1',index=False,header=False,startrow=1)
        writer.save() 
    
    
    
    
# Execution    
mainExecution()

https://loker.bnpb.go.id/s/GugusTugasCovid19?path=%2FData%20Kemenkes%2FLaporan%20Media
Number of results Info File : 212
Number of results Info Date : 212
August 2, 2020 3:28 PM
Yes It is, Please Download this file : 1 AGUSTUS 2020__LINELIST KASUS POSITIF.pdf
July 1, 2020 4:40 PM
Yes It is, Please Download this file : 1 JULI 2020 LAPORAN MEDIA COVID 19.pdf
October 1, 2020 9:08 PM
Yes It is, Please Download this file : 1 Oktober 2020__TABEL RILIS COVID-19.pdf
September 6, 2020 1:43 PM
Yes It is, Please Download this file : 1 SEPTEMBER 2020_LINELIST KASUS POSITIF.pdf
August 2, 2020 2:36 PM
Tidak Terdownload, karena file sudah ada
October 3, 2020 3:36 PM
Tidak Terdownload, karena file sudah ada
September 6, 2020 1:42 PM
Tidak Terdownload, karena file sudah ada
August 3, 2020 2:51 PM
Tidak Terdownload, karena file sudah ada
July 3, 2020 3:50 PM
Tidak Terdownload, karena file sudah ada
October 3, 2020 3:02 PM
Tidak Terdownload, karena file sudah ada
September 6, 2020 1:42 PM
Tidak Terdownlo

May 4, 2020 4:14 PM
Tidak Terdownload, karena file sudah ada
April 6, 2020 7:16 PM
Tidak Terdownload, karena file sudah ada
June 5, 2020 2:46 PM
Tidak Terdownload, karena file sudah ada
May 5, 2020 2:27 PM
Tidak Terdownload, karena file sudah ada
July 6, 2020 2:40 PM
Tidak Terdownload, karena file sudah ada
July 6, 2020 2:26 PM
Tidak Terdownload, karena file sudah ada
June 7, 2020 3:50 PM
Tidak Terdownload, karena file sudah ada
May 6, 2020 2:40 PM
Tidak Terdownload, karena file sudah ada
April 7, 2020 2:27 PM
Tidak Terdownload, karena file sudah ada
July 7, 2020 2:41 PM
Tidak Terdownload, karena file sudah ada
June 7, 2020 2:15 PM
Tidak Terdownload, karena file sudah ada
April 8, 2020 8:45 PM
Tidak Terdownload, karena file sudah ada
June 8, 2020 2:49 PM
Tidak Terdownload, karena file sudah ada
May 8, 2020 2:47 PM
Tidak Terdownload, karena file sudah ada
April 9, 2020 3:09 PM
Tidak Terdownload, karena file sudah ada
June 9, 2020 2:22 PM
Tidak Terdownload, karena file sudah ada
May 20, 

NameError: name 'name_pdf' is not defined